<!--
Copyright (c) 2025 Milin Patel
Hochschule Kempten - University of Applied Sciences

Autonomous Driving: AI Safety and Security Workshop
This project is licensed under the MIT License.
See LICENSE file in the root directory for full license text.
-->

*Copyright © 2025 Milin Patel. All Rights Reserved.*

# ISO 21448: Safety of the Intended Functionality (SOTIF)

[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/milinpatel07/Autonomous-Driving_AI-Safety-and-Security/blob/main/AV_Perception_Safety_Workshop/Session_3_Safety_and_Security_Standards/notebooks/12_ISO_21448_SOTIF.ipynb)

**Duration:** 30 minutes

## Learning Objectives
- Understand SOTIF principles and how they differ from ISO 26262
- Master the four scenario categories
- Identify triggering conditions and performance limitations
- Apply SOTIF methodology to ML-based perception systems
- Integrate SOTIF with functional safety

---

In [ ]:
# Setup: Install and import required libraries
import sys

# Install packages if in Colab
if 'google.colab' in sys.modules:
    !pip install -q matplotlib pandas numpy seaborn plotly

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from matplotlib.patches import Rectangle, FancyBboxPatch
import warnings
warnings.filterwarnings('ignore')

# Set style
plt.style.use('seaborn-v0_8-darkgrid')
sns.set_palette("husl")

print("✓ Setup complete!")

## 1. SOTIF Overview

### What is SOTIF?

**ISO/PAS 21448** (Safety of the Intended Functionality) addresses risks resulting from:
- **Performance limitations** of the intended functionality
- **Reasonably foreseeable misuse** by users

### Key Insight

💡 **Even when the system works as designed (no faults), it can still be unsafe!**

### SOTIF vs ISO 26262

| Aspect | ISO 26262 | ISO 21448 (SOTIF) |
|--------|-----------|-------------------|
| **Focus** | Random HW faults, systematic failures | Performance limitations, unknowns |
| **Scope** | Malfunctioning behavior | Insufficient performance |
| **Root cause** | Component failure | Design limitation, unknown scenarios |
| **Approach** | Fault avoidance/tolerance | Scenario coverage, validation |
| **Metrics** | ASIL, FIT rates | Scenario coverage |
| **Example** | Sensor hardware failure | Sensor can't detect dark pedestrians |

### Why SOTIF Matters for Autonomous Driving

1. **Complex perception algorithms**: ML models have inherent limitations
2. **Infinite scenario space**: Can't test all possible situations
3. **Environmental variability**: Weather, lighting, road conditions
4. **Unknown unknowns**: Scenarios we haven't thought of yet

In [ ]:
# Visualize ISO 26262 vs SOTIF coverage
def visualize_safety_coverage():
    """
    Visualize the complementary coverage of ISO 26262 and SOTIF
    """
    fig, ax = plt.subplots(figsize=(14, 8))
    
    # ISO 26262 coverage (left circle)
    circle1 = plt.Circle((0.35, 0.5), 0.25, color='#3498db', alpha=0.6, label='ISO 26262')
    ax.add_patch(circle1)
    
    # SOTIF coverage (right circle)
    circle2 = plt.Circle((0.65, 0.5), 0.25, color='#e74c3c', alpha=0.6, label='ISO 21448 (SOTIF)')
    ax.add_patch(circle2)
    
    # Labels
    ax.text(0.25, 0.5, 'Random HW\nFailures\n\nSystematic\nSW Faults\n\nASIL-based\nRequirements',
            ha='center', va='center', fontsize=11, fontweight='bold')
    
    ax.text(0.75, 0.5, 'Performance\nLimitations\n\nUnknown\nScenarios\n\nValidation\nGaps',
            ha='center', va='center', fontsize=11, fontweight='bold')
    
    ax.text(0.5, 0.5, 'Both\naddress\nsafety',
            ha='center', va='center', fontsize=10, fontweight='bold', color='white',
            bbox=dict(boxstyle='round', facecolor='purple', alpha=0.8))
    
    # Examples below
    ax.text(0.25, 0.15, 'Example:\nCamera sensor\nelectrical failure',
            ha='center', va='center', fontsize=10,
            bbox=dict(boxstyle='round', facecolor='lightblue', alpha=0.7))
    
    ax.text(0.75, 0.15, 'Example:\nCamera cannot detect\ndark objects at night',
            ha='center', va='center', fontsize=10,
            bbox=dict(boxstyle='round', facecolor='lightcoral', alpha=0.7))
    
    ax.set_xlim(0, 1)
    ax.set_ylim(0, 1)
    ax.axis('off')
    ax.set_title('ISO 26262 vs ISO 21448: Complementary Safety Approaches', 
                fontsize=16, fontweight='bold', pad=20)
    ax.legend(loc='upper center', fontsize=12, ncol=2)
    
    plt.tight_layout()
    plt.show()

visualize_safety_coverage()

## 2. Four Scenario Categories

SOTIF defines four categories based on two dimensions:
- **Known vs Unknown**: Whether the scenario is identified
- **Safe vs Unsafe**: Whether the system performs adequately

### The SOTIF Space

```
                    Known              Unknown
            ┌─────────────────┬─────────────────┐
            │                 │                 │
   Safe     │   Known Safe    │  Unknown Safe   │
            │     (Target)    │   (To Discover) │
            │                 │                 │
            ├─────────────────┼─────────────────┤
            │                 │                 │
   Unsafe   │  Known Unsafe   │ Unknown Unsafe  │
            │   (Mitigate!)   │   (High Risk!)  │
            │                 │                 │
            └─────────────────┴─────────────────┘
```

### Category Definitions

1. **Known Safe (S1)**: Scenarios where system performs acceptably
   - Well-tested, validated scenarios
   - Within Operational Design Domain (ODD)
   - Example: Detecting pedestrians in good visibility

2. **Known Unsafe (S2)**: Known scenarios where system is insufficient
   - Identified limitations
   - Requires mitigation or ODD restriction
   - Example: Cannot detect pedestrians in heavy fog

3. **Unknown Safe (S3)**: Unidentified scenarios where system would perform well
   - Need to discover and verify
   - Expand validation coverage
   - Example: Untested but safe edge cases

4. **Unknown Unsafe (S4)**: Unidentified scenarios with insufficient performance
   - **Highest risk!**
   - Goal: Minimize this region through extensive testing
   - Example: Adversarial scenarios we haven't encountered

In [ ]:
# Visualize SOTIF scenario space
def visualize_sotif_space():
    """
    Visualize the four SOTIF scenario categories
    """
    fig, ax = plt.subplots(figsize=(12, 10))
    
    # Define quadrants
    quadrants = [
        {'name': 'S1: Known Safe', 'color': '#2ecc71', 'x': 0.25, 'y': 0.75,
         'description': 'Validated scenarios\nwithin ODD\n\nTarget operating region\n\nExample: Day, clear weather,\nwell-marked lanes'},
        {'name': 'S2: Known Unsafe', 'color': '#e74c3c', 'x': 0.25, 'y': 0.25,
         'description': 'Identified limitations\n\nRequires mitigation\nor ODD restriction\n\nExample: Heavy fog,\nno lane markings'},
        {'name': 'S3: Unknown Safe', 'color': '#f39c12', 'x': 0.75, 'y': 0.75,
         'description': 'Untested scenarios\n(likely safe)\n\nExpand validation\n\nExample: Novel but\nbenign conditions'},
        {'name': 'S4: Unknown Unsafe', 'color': '#c0392b', 'x': 0.75, 'y': 0.25,
         'description': 'HIGHEST RISK\n\nUnknown failures\n\nMinimize through\ntesting\n\nExample: Adversarial\nscenarios'}
    ]
    
    for quad in quadrants:
        rect = FancyBboxPatch((quad['x']-0.22, quad['y']-0.22), 0.44, 0.44,
                             boxstyle="round,pad=0.02", 
                             facecolor=quad['color'], edgecolor='black', 
                             linewidth=3, alpha=0.7)
        ax.add_patch(rect)
        
        ax.text(quad['x'], quad['y']+0.15, quad['name'], 
               ha='center', va='center', fontsize=14, fontweight='bold',
               bbox=dict(boxstyle='round', facecolor='white', alpha=0.9))
        
        ax.text(quad['x'], quad['y']-0.08, quad['description'],
               ha='center', va='center', fontsize=9, style='italic')
    
    # Axes labels
    ax.text(0.5, 0.95, 'SAFE', ha='center', va='center', 
           fontsize=16, fontweight='bold', color='green')
    ax.text(0.5, 0.05, 'UNSAFE', ha='center', va='center',
           fontsize=16, fontweight='bold', color='red')
    ax.text(0.05, 0.5, 'KNOWN', ha='center', va='center', rotation=90,
           fontsize=16, fontweight='bold', color='blue')
    ax.text(0.95, 0.5, 'UNKNOWN', ha='center', va='center', rotation=90,
           fontsize=16, fontweight='bold', color='orange')
    
    # Center lines
    ax.axhline(y=0.5, color='black', linewidth=3, linestyle='--')
    ax.axvline(x=0.5, color='black', linewidth=3, linestyle='--')
    
    # Arrows showing SOTIF process
    ax.annotate('', xy=(0.52, 0.75), xytext=(0.72, 0.75),
               arrowprops=dict(arrowstyle='<-', lw=3, color='purple'))
    ax.text(0.62, 0.80, 'Discover', ha='center', fontsize=10, fontweight='bold', color='purple')
    
    ax.annotate('', xy=(0.25, 0.48), xytext=(0.25, 0.28),
               arrowprops=dict(arrowstyle='<-', lw=3, color='purple'))
    ax.text(0.32, 0.38, 'Identify', ha='center', fontsize=10, fontweight='bold', 
           color='purple', rotation=90)
    
    ax.annotate('', xy=(0.48, 0.25), xytext=(0.72, 0.25),
               arrowprops=dict(arrowstyle='<-', lw=3, color='purple'))
    ax.text(0.60, 0.20, 'Minimize!', ha='center', fontsize=10, fontweight='bold', color='purple')
    
    ax.set_xlim(0, 1)
    ax.set_ylim(0, 1)
    ax.axis('off')
    ax.set_title('SOTIF Scenario Space: Four Categories', 
                fontsize=18, fontweight='bold', pad=20)
    
    plt.tight_layout()
    plt.show()

visualize_sotif_space()

## 3. Triggering Conditions and Performance Limitations

### Triggering Conditions

Circumstances that can lead to hazardous behavior:

1. **Sensor Limitations**
   - Range limitations
   - Resolution constraints
   - Environmental conditions (rain, fog, glare)
   - Occlusions

2. **Algorithm Limitations**
   - Out-of-distribution inputs
   - Edge cases in training data
   - Processing latency
   - False positives/negatives

3. **Environmental Factors**
   - Unusual road geometry
   - Unexpected objects
   - Rare weather conditions
   - Complex traffic situations

4. **Human Factors**
   - Misuse or overreliance
   - Unexpected driver intervention
   - Distraction during takeover

In [ ]:
# Example: Triggering conditions for pedestrian detection
def analyze_triggering_conditions():
    """
    Analyze triggering conditions for pedestrian detection system
    """
    # Define triggering conditions and their impact
    triggering_conditions = {
        'Environmental': {
            'Heavy rain': {'detection_rate': 0.75, 'severity': 'High'},
            'Dense fog': {'detection_rate': 0.60, 'severity': 'Critical'},
            'Direct sunlight glare': {'detection_rate': 0.80, 'severity': 'Medium'},
            'Night (no street lights)': {'detection_rate': 0.70, 'severity': 'High'},
            'Snow/ice on sensor': {'detection_rate': 0.50, 'severity': 'Critical'}
        },
        'Scenario Complexity': {
            'Crowded intersection': {'detection_rate': 0.85, 'severity': 'Medium'},
            'Pedestrian behind vehicle': {'detection_rate': 0.65, 'severity': 'High'},
            'Child running suddenly': {'detection_rate': 0.78, 'severity': 'High'},
            'Pedestrian with unusual pose': {'detection_rate': 0.82, 'severity': 'Medium'},
            'Multiple occlusions': {'detection_rate': 0.70, 'severity': 'High'}
        },
        'Object Characteristics': {
            'Dark clothing at night': {'detection_rate': 0.72, 'severity': 'High'},
            'Reflective clothing': {'detection_rate': 0.95, 'severity': 'Low'},
            'Unusual pedestrian size': {'detection_rate': 0.80, 'severity': 'Medium'},
            'Pedestrian with large object': {'detection_rate': 0.77, 'severity': 'Medium'},
            'Partially occluded': {'detection_rate': 0.75, 'severity': 'High'}
        }
    }
    
    # Create DataFrame
    data = []
    for category, conditions in triggering_conditions.items():
        for condition, metrics in conditions.items():
            data.append({
                'Category': category,
                'Triggering Condition': condition,
                'Detection Rate': metrics['detection_rate'],
                'Severity': metrics['severity']
            })
    
    df = pd.DataFrame(data)
    
    # Visualize
    fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(16, 6))
    
    # Detection rate by category
    category_avg = df.groupby('Category')['Detection Rate'].mean().sort_values()
    colors = ['#e74c3c' if x < 0.80 else '#f39c12' if x < 0.90 else '#2ecc71' 
             for x in category_avg.values]
    category_avg.plot(kind='barh', ax=ax1, color=colors, edgecolor='black', linewidth=2)
    ax1.set_xlabel('Average Detection Rate', fontsize=12, fontweight='bold')
    ax1.set_title('Detection Performance by Triggering Condition Category', 
                 fontsize=13, fontweight='bold')
    ax1.axvline(x=0.95, color='green', linestyle='--', linewidth=2, label='Target (95%)')
    ax1.axvline(x=0.85, color='orange', linestyle='--', linewidth=2, label='Warning (85%)')
    ax1.legend()
    ax1.grid(axis='x', alpha=0.3)
    
    # Scatter plot: detection rate vs severity
    severity_map = {'Low': 1, 'Medium': 2, 'High': 3, 'Critical': 4}
    df['Severity_Numeric'] = df['Severity'].map(severity_map)
    
    for category in df['Category'].unique():
        subset = df[df['Category'] == category]
        ax2.scatter(subset['Detection Rate'], subset['Severity_Numeric'], 
                   s=200, alpha=0.6, label=category, edgecolor='black', linewidth=2)
    
    # Highlight critical region (low detection + high severity)
    ax2.axhspan(3, 4.5, xmin=0, xmax=0.35, alpha=0.2, color='red', label='Critical Region')
    
    ax2.set_xlabel('Detection Rate', fontsize=12, fontweight='bold')
    ax2.set_ylabel('Severity', fontsize=12, fontweight='bold')
    ax2.set_yticks([1, 2, 3, 4])
    ax2.set_yticklabels(['Low', 'Medium', 'High', 'Critical'])
    ax2.set_title('Risk Matrix: Detection Rate vs Severity', fontsize=13, fontweight='bold')
    ax2.legend(loc='lower right')
    ax2.grid(alpha=0.3)
    ax2.set_xlim(0.45, 1.0)
    
    plt.tight_layout()
    plt.show()
    
    return df

df_triggers = analyze_triggering_conditions()

print("\n" + "="*80)
print("Triggering Conditions Analysis")
print("="*80)
display(df_triggers.sort_values('Detection Rate'))

## 4. SOTIF Process: Design, Verification, Validation, Field Monitoring

### The SOTIF V-Model

SOTIF follows an iterative process:

1. **Specify and Design**
   - Define intended functionality
   - Identify known limitations
   - Establish ODD (Operational Design Domain)
   - Design mitigation measures

2. **Verification Activities**
   - Analyze triggering conditions
   - Identify known unsafe scenarios
   - Test against known scenarios
   - Verify mitigation effectiveness

3. **Validation Activities**
   - Discover unknown scenarios
   - Test in diverse conditions
   - Expand scenario database
   - Reduce unknown unsafe space

4. **Field Monitoring**
   - Monitor real-world performance
   - Identify emerging scenarios
   - Collect edge cases
   - Trigger updates/recalls if needed

In [ ]:
# Visualize SOTIF process
def visualize_sotif_process():
    """
    Visualize the iterative SOTIF process
    """
    fig, ax = plt.subplots(figsize=(14, 10))
    
    # Process steps
    steps = [
        {'name': '1. Specify\nIntended\nFunctionality', 'x': 0.15, 'y': 0.85, 'color': '#3498db'},
        {'name': '2. Identify\nKnown\nLimitations', 'x': 0.45, 'y': 0.85, 'color': '#e74c3c'},
        {'name': '3. Design\nMitigation', 'x': 0.75, 'y': 0.85, 'color': '#2ecc71'},
        {'name': '4. Verification\nTesting', 'x': 0.85, 'y': 0.65, 'color': '#f39c12'},
        {'name': '5. Validation\nTesting', 'x': 0.85, 'y': 0.40, 'color': '#9b59b6'},
        {'name': '6. Field\nMonitoring', 'x': 0.75, 'y': 0.20, 'color': '#1abc9c'},
        {'name': '7. Scenario\nDatabase\nUpdate', 'x': 0.45, 'y': 0.20, 'color': '#34495e'},
        {'name': '8. Iterate/\nImprove', 'x': 0.15, 'y': 0.20, 'color': '#e67e22'}
    ]
    
    # Draw process flow
    for i, step in enumerate(steps):
        rect = FancyBboxPatch((step['x']-0.08, step['y']-0.08), 0.16, 0.12,
                             boxstyle="round,pad=0.01",
                             facecolor=step['color'], edgecolor='black',
                             linewidth=2.5, alpha=0.8)
        ax.add_patch(rect)
        ax.text(step['x'], step['y'], step['name'],
               ha='center', va='center', fontsize=10, fontweight='bold', color='white')
        
        # Draw arrows
        if i < len(steps) - 1:
            next_step = steps[i + 1]
            ax.annotate('', xy=(next_step['x']-0.08, next_step['y']), 
                       xytext=(step['x']+0.08, step['y']),
                       arrowprops=dict(arrowstyle='->', lw=3, color='black'))
    
    # Closing the loop
    ax.annotate('', xy=(steps[0]['x']-0.08, steps[0]['y']-0.08), 
               xytext=(steps[-1]['x'], steps[-1]['y']+0.08),
               arrowprops=dict(arrowstyle='->', lw=3, color='purple', linestyle='dashed'))
    ax.text(0.15, 0.50, 'Continuous\nImprovement\nLoop', ha='center', va='center',
           fontsize=11, fontweight='bold', color='purple',
           bbox=dict(boxstyle='round', facecolor='yellow', alpha=0.6))
    
    # Key activities boxes
    activities = [
        {'text': 'Known → Safe\n(S2 → S1)', 'x': 0.30, 'y': 0.65},
        {'text': 'Unknown → Known\n(S3/S4 → S1/S2)', 'x': 0.60, 'y': 0.50},
        {'text': 'Minimize S4\n(Unknown Unsafe)', 'x': 0.30, 'y': 0.35}
    ]
    
    for activity in activities:
        ax.text(activity['x'], activity['y'], activity['text'],
               ha='center', va='center', fontsize=9, style='italic',
               bbox=dict(boxstyle='round', facecolor='lightyellow', 
                        edgecolor='orange', linewidth=2, alpha=0.8))
    
    ax.set_xlim(0, 1)
    ax.set_ylim(0.1, 1)
    ax.axis('off')
    ax.set_title('SOTIF Process: Iterative Safety Validation', 
                fontsize=16, fontweight='bold', pad=20)
    
    plt.tight_layout()
    plt.show()

visualize_sotif_process()

## 5. Example: SOTIF Analysis for ML-Based Perception

Let's perform a complete SOTIF analysis for a pedestrian detection system.

In [ ]:
# Complete SOTIF analysis example
class SOTIFAnalysis:
    """
    SOTIF analysis framework for autonomous driving systems
    """
    
    def __init__(self, system_name: str, odd: str):
        self.system_name = system_name
        self.odd = odd
        self.scenarios = []
    
    def add_scenario(self, scenario_id: str, description: str, 
                    category: str, performance: float, 
                    triggering_condition: str, mitigation: str):
        """
        Add a scenario to the SOTIF analysis
        """
        scenario = {
            'id': scenario_id,
            'description': description,
            'category': category,  # S1, S2, S3, S4
            'performance': performance,  # 0-1
            'triggering_condition': triggering_condition,
            'mitigation': mitigation
        }
        self.scenarios.append(scenario)
    
    def analyze(self):
        """
        Analyze scenario distribution
        """
        df = pd.DataFrame(self.scenarios)
        
        # Count by category
        category_counts = df['category'].value_counts()
        
        # Average performance by category
        category_perf = df.groupby('category')['performance'].mean()
        
        return df, category_counts, category_perf

# Example: Pedestrian Detection System
sotif = SOTIFAnalysis(
    system_name="Pedestrian Detection with Emergency Braking",
    odd="Urban/suburban roads, daylight, speeds up to 60 km/h"
)

# Add scenarios
sotif.add_scenario(
    scenario_id='SC-001',
    description='Adult pedestrian crossing, daylight, clear weather',
    category='S1',
    performance=0.98,
    triggering_condition='None - within ODD',
    mitigation='N/A - performs adequately'
)

sotif.add_scenario(
    scenario_id='SC-002',
    description='Pedestrian in heavy rain',
    category='S2',
    performance=0.75,
    triggering_condition='Camera degradation in rain',
    mitigation='Add radar sensor, reduce speed, driver warning'
)

sotif.add_scenario(
    scenario_id='SC-003',
    description='Pedestrian with reflective vest, dusk',
    category='S3',
    performance=0.96,
    triggering_condition='Low light but reflective clothing',
    mitigation='Validate and move to S1'
)

sotif.add_scenario(
    scenario_id='SC-004',
    description='Dense fog, pedestrian at crosswalk',
    category='S2',
    performance=0.60,
    triggering_condition='Sensor range limitation',
    mitigation='Restrict ODD or require LIDAR'
)

sotif.add_scenario(
    scenario_id='SC-005',
    description='Child on skateboard, sudden entry',
    category='S2',
    performance=0.70,
    triggering_condition='Unusual pose + high speed',
    mitigation='Expand training data, improve model'
)

sotif.add_scenario(
    scenario_id='SC-006',
    description='Pedestrian behind construction barrier',
    category='S4',
    performance=0.45,
    triggering_condition='Occlusion + unexpected location',
    mitigation='HIGH PRIORITY: Add to test scenarios, improve detection'
)

sotif.add_scenario(
    scenario_id='SC-007',
    description='Pedestrian with large umbrella',
    category='S3',
    performance=0.92,
    triggering_condition='Unusual object shape',
    mitigation='Test and validate'
)

sotif.add_scenario(
    scenario_id='SC-008',
    description='Group of pedestrians at night',
    category='S2',
    performance=0.78,
    triggering_condition='Low light + clustering',
    mitigation='Improved night vision, speed reduction'
)

# Analyze
df_sotif, counts, perf = sotif.analyze()

print("\n" + "="*80)
print(f"SOTIF Analysis: {sotif.system_name}")
print(f"ODD: {sotif.odd}")
print("="*80)
display(df_sotif[['id', 'description', 'category', 'performance', 'triggering_condition']])

# Visualize distribution
fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(16, 6))

# Scenario distribution
colors_cat = {'S1': '#2ecc71', 'S2': '#e74c3c', 'S3': '#f39c12', 'S4': '#c0392b'}
counts_ordered = counts.reindex(['S1', 'S2', 'S3', 'S4'], fill_value=0)
bars1 = counts_ordered.plot(kind='bar', ax=ax1, 
                            color=[colors_cat[x] for x in counts_ordered.index],
                            edgecolor='black', linewidth=2)
ax1.set_title('Scenario Distribution by SOTIF Category', fontsize=13, fontweight='bold')
ax1.set_xlabel('SOTIF Category', fontsize=12, fontweight='bold')
ax1.set_ylabel('Number of Scenarios', fontsize=12, fontweight='bold')
ax1.set_xticklabels(['S1\nKnown Safe', 'S2\nKnown Unsafe', 'S3\nUnknown Safe', 'S4\nUnknown Unsafe'], 
                   rotation=0)
ax1.grid(axis='y', alpha=0.3)

# Performance by category
perf_ordered = perf.reindex(['S1', 'S2', 'S3', 'S4'], fill_value=0)
bars2 = perf_ordered.plot(kind='bar', ax=ax2,
                          color=[colors_cat[x] for x in perf_ordered.index],
                          edgecolor='black', linewidth=2)
ax2.axhline(y=0.95, color='green', linestyle='--', linewidth=2, label='Target Performance')
ax2.axhline(y=0.80, color='orange', linestyle='--', linewidth=2, label='Minimum Acceptable')
ax2.set_title('Average Performance by SOTIF Category', fontsize=13, fontweight='bold')
ax2.set_xlabel('SOTIF Category', fontsize=12, fontweight='bold')
ax2.set_ylabel('Average Performance', fontsize=12, fontweight='bold')
ax2.set_xticklabels(['S1\nKnown Safe', 'S2\nKnown Unsafe', 'S3\nUnknown Safe', 'S4\nUnknown Unsafe'],
                   rotation=0)
ax2.set_ylim(0, 1.05)
ax2.legend()
ax2.grid(axis='y', alpha=0.3)

plt.tight_layout()
plt.show()

# Recommendations
print("\n" + "="*80)
print("SOTIF Recommendations")
print("="*80)
print(f"\n1. Known Safe (S1): {counts.get('S1', 0)} scenarios - Continue validation")
print(f"2. Known Unsafe (S2): {counts.get('S2', 0)} scenarios - Implement mitigations")
print(f"3. Unknown Safe (S3): {counts.get('S3', 0)} scenarios - Test and validate")
print(f"4. Unknown Unsafe (S4): {counts.get('S4', 0)} scenarios - CRITICAL! Expand testing")

if counts.get('S4', 0) > 0:
    print("\n⚠️  HIGH PRIORITY: Unknown Unsafe scenarios identified!")
    print("   Action: Expand scenario database and validation testing")

s2_scenarios = df_sotif[df_sotif['category'] == 'S2']
if len(s2_scenarios) > 0:
    print(f"\n⚠️  {len(s2_scenarios)} Known Unsafe scenarios require mitigation:")
    for _, row in s2_scenarios.iterrows():
        print(f"   - {row['id']}: {row['mitigation']}")

## 6. Integration with ISO 26262

### Complementary Application

ISO 26262 and SOTIF work together:

| Aspect | ISO 26262 | ISO 21448 (SOTIF) |
|--------|-----------|-------------------|
| **When to apply** | Always (E/E systems) | Complex systems with AI/ML |
| **Safety goals** | Derived from HARA | Derived from triggering conditions |
| **Requirements** | ASIL-based | Performance-based |
| **Verification** | Fault injection, FIT | Scenario coverage |
| **Validation** | System testing | Field testing, edge case discovery |
| **Lifecycle** | Design to decommissioning | Includes field monitoring |

### Combined Workflow

1. **Perform HARA** (ISO 26262) → Identify fault-related hazards
2. **Perform SOTIF analysis** → Identify performance limitations
3. **Define combined safety goals** → Address both fault and performance
4. **Design system** → Implement fault tolerance AND performance robustness
5. **Verify** → Test for faults AND scenarios
6. **Validate** → System testing AND field monitoring
7. **Monitor in field** → Detect both new faults AND emerging scenarios

In [ ]:
# Visualize ISO 26262 + SOTIF integration
def visualize_integration():
    """
    Show how ISO 26262 and SOTIF integrate
    """
    fig, ax = plt.subplots(figsize=(14, 10))
    
    # ISO 26262 path (left)
    iso_steps = [
        {'y': 0.9, 'text': 'Item Definition\n& HARA', 'color': '#3498db'},
        {'y': 0.7, 'text': 'Safety Goals\n(fault-based)', 'color': '#3498db'},
        {'y': 0.5, 'text': 'Functional\nSafety Req.', 'color': '#3498db'},
        {'y': 0.3, 'text': 'Fault Injection\nTesting', 'color': '#3498db'},
        {'y': 0.1, 'text': 'ASIL\nVerification', 'color': '#3498db'}
    ]
    
    # SOTIF path (right)
    sotif_steps = [
        {'y': 0.9, 'text': 'ODD Definition\n& Limitations', 'color': '#e74c3c'},
        {'y': 0.7, 'text': 'Triggering\nConditions', 'color': '#e74c3c'},
        {'y': 0.5, 'text': 'Performance\nRequirements', 'color': '#e74c3c'},
        {'y': 0.3, 'text': 'Scenario-based\nTesting', 'color': '#e74c3c'},
        {'y': 0.1, 'text': 'Coverage\nValidation', 'color': '#e74c3c'}
    ]
    
    x_iso = 0.25
    x_sotif = 0.75
    
    # Draw ISO 26262 path
    for i, step in enumerate(iso_steps):
        rect = FancyBboxPatch((x_iso-0.10, step['y']-0.05), 0.20, 0.10,
                             boxstyle="round,pad=0.01",
                             facecolor=step['color'], edgecolor='black',
                             linewidth=2, alpha=0.8)
        ax.add_patch(rect)
        ax.text(x_iso, step['y'], step['text'], ha='center', va='center',
               fontsize=10, fontweight='bold', color='white')
        
        if i < len(iso_steps) - 1:
            ax.arrow(x_iso, step['y']-0.06, 0, -0.08, 
                    head_width=0.03, head_length=0.02, fc='black', ec='black', linewidth=2)
    
    # Draw SOTIF path
    for i, step in enumerate(sotif_steps):
        rect = FancyBboxPatch((x_sotif-0.10, step['y']-0.05), 0.20, 0.10,
                             boxstyle="round,pad=0.01",
                             facecolor=step['color'], edgecolor='black',
                             linewidth=2, alpha=0.8)
        ax.add_patch(rect)
        ax.text(x_sotif, step['y'], step['text'], ha='center', va='center',
               fontsize=10, fontweight='bold', color='white')
        
        if i < len(sotif_steps) - 1:
            ax.arrow(x_sotif, step['y']-0.06, 0, -0.08,
                    head_width=0.03, head_length=0.02, fc='black', ec='black', linewidth=2)
    
    # Integration points
    integrations = [
        {'y': 0.8, 'text': 'Combined\nHazard Analysis'},
        {'y': 0.6, 'text': 'Unified\nSafety Goals'},
        {'y': 0.4, 'text': 'Integrated\nRequirements'},
        {'y': 0.2, 'text': 'Comprehensive\nTesting'},
        {'y': 0.0, 'text': 'Complete\nSafety Case'}
    ]
    
    for integration in integrations:
        # Draw connection
        ax.plot([x_iso+0.10, x_sotif-0.10], [integration['y'], integration['y']],
               'g--', linewidth=2.5, alpha=0.6)
        
        # Integration label
        ax.text(0.5, integration['y'], integration['text'],
               ha='center', va='center', fontsize=9, fontweight='bold',
               bbox=dict(boxstyle='round', facecolor='lightgreen', 
                        edgecolor='darkgreen', linewidth=2, alpha=0.8))
    
    # Labels
    ax.text(x_iso, 0.98, 'ISO 26262\n(Functional Safety)',
           ha='center', va='top', fontsize=13, fontweight='bold',
           bbox=dict(boxstyle='round', facecolor='lightblue', edgecolor='blue', linewidth=2))
    
    ax.text(x_sotif, 0.98, 'ISO 21448\n(SOTIF)',
           ha='center', va='top', fontsize=13, fontweight='bold',
           bbox=dict(boxstyle='round', facecolor='lightcoral', edgecolor='red', linewidth=2))
    
    ax.set_xlim(0, 1)
    ax.set_ylim(-0.05, 1.05)
    ax.axis('off')
    ax.set_title('Integration of ISO 26262 and ISO 21448 (SOTIF)', 
                fontsize=16, fontweight='bold', pad=20)
    
    plt.tight_layout()
    plt.show()

visualize_integration()

## 7. Validation Strategies: Scenario-Based Testing

### Scenario Generation Approaches

1. **Knowledge-based**: Expert-defined scenarios
2. **Data-driven**: Scenarios from real-world data
3. **Simulation-based**: Generated scenarios in virtual environments
4. **Combinatorial**: Parameter space exploration
5. **Adversarial**: Intentionally challenging scenarios

### Validation Metrics

- **Scenario coverage**: % of identified scenarios tested
- **Performance metrics**: Detection rate, false positives, latency
- **Edge case discovery rate**: New scenarios found per test hour
- **ODD compliance**: % operation within defined ODD

In [ ]:
# Simulate scenario coverage over time
def simulate_scenario_coverage():
    """
    Simulate scenario discovery and coverage improvement over time
    """
    np.random.seed(42)
    
    # Time in weeks
    weeks = np.arange(0, 52, 1)
    
    # Known scenarios (discovered over time)
    known_safe = 50 + 150 * (1 - np.exp(-weeks/10))
    known_unsafe = 20 + 80 * (1 - np.exp(-weeks/15))
    
    # Unknown scenarios (decreasing over time)
    unknown_safe = 100 * np.exp(-weeks/12)
    unknown_unsafe = 50 * np.exp(-weeks/8)
    
    # Total scenarios
    total = known_safe + known_unsafe + unknown_safe + unknown_unsafe
    
    # Coverage percentage
    coverage = (known_safe + known_unsafe) / total * 100
    
    # Plot
    fig, (ax1, ax2) = plt.subplots(2, 1, figsize=(14, 10))
    
    # Stacked area plot
    ax1.fill_between(weeks, 0, known_safe, label='S1: Known Safe', color='#2ecc71', alpha=0.7)
    ax1.fill_between(weeks, known_safe, known_safe + known_unsafe, 
                    label='S2: Known Unsafe', color='#e74c3c', alpha=0.7)
    ax1.fill_between(weeks, known_safe + known_unsafe, 
                    known_safe + known_unsafe + unknown_safe,
                    label='S3: Unknown Safe', color='#f39c12', alpha=0.7)
    ax1.fill_between(weeks, known_safe + known_unsafe + unknown_safe, total,
                    label='S4: Unknown Unsafe', color='#c0392b', alpha=0.7)
    
    ax1.set_xlabel('Validation Time (weeks)', fontsize=12, fontweight='bold')
    ax1.set_ylabel('Number of Scenarios', fontsize=12, fontweight='bold')
    ax1.set_title('SOTIF Scenario Discovery Over Time', fontsize=14, fontweight='bold')
    ax1.legend(loc='right')
    ax1.grid(alpha=0.3)
    
    # Coverage plot
    ax2.plot(weeks, coverage, linewidth=3, color='#9b59b6', label='Scenario Coverage')
    ax2.axhline(y=95, color='green', linestyle='--', linewidth=2, label='Target (95%)')
    ax2.axhline(y=90, color='orange', linestyle='--', linewidth=2, label='Minimum (90%)')
    ax2.fill_between(weeks, 95, 100, alpha=0.2, color='green')
    ax2.fill_between(weeks, 90, 95, alpha=0.2, color='yellow')
    ax2.fill_between(weeks, 0, 90, alpha=0.2, color='red')
    
    ax2.set_xlabel('Validation Time (weeks)', fontsize=12, fontweight='bold')
    ax2.set_ylabel('Coverage (%)', fontsize=12, fontweight='bold')
    ax2.set_title('Scenario Coverage Improvement', fontsize=14, fontweight='bold')
    ax2.legend()
    ax2.grid(alpha=0.3)
    ax2.set_ylim(0, 100)
    
    plt.tight_layout()
    plt.show()
    
    # Analysis
    print("\n" + "="*80)
    print("Scenario Coverage Analysis")
    print("="*80)
    print(f"\nWeek 0: {coverage[0]:.1f}% coverage")
    print(f"Week 26: {coverage[26]:.1f}% coverage")
    print(f"Week 52: {coverage[-1]:.1f}% coverage")
    
    week_95 = np.where(coverage >= 95)[0]
    if len(week_95) > 0:
        print(f"\n✓ Target coverage (95%) reached at week {week_95[0]}")
    else:
        print(f"\n⚠️  Target coverage (95%) not reached within 52 weeks")
    
    print(f"\nFinal scenario distribution:")
    print(f"  S1 (Known Safe): {int(known_safe[-1])} scenarios")
    print(f"  S2 (Known Unsafe): {int(known_unsafe[-1])} scenarios")
    print(f"  S3 (Unknown Safe): {int(unknown_safe[-1])} scenarios")
    print(f"  S4 (Unknown Unsafe): {int(unknown_unsafe[-1])} scenarios")

simulate_scenario_coverage()

## 8. Summary and Key Takeaways

### SOTIF in a Nutshell

✓ **Performance focus**: Addresses limitations even when system works as designed  
✓ **Scenario-centric**: Coverage of operational scenarios is key  
✓ **Unknown unknowns**: Main goal is to minimize Unknown Unsafe space (S4)  
✓ **Iterative process**: Continuous discovery, validation, and improvement  
✓ **Complements ISO 26262**: Together they provide comprehensive safety coverage  

### Critical Success Factors

1. **Comprehensive ODD definition**: Clearly define operational boundaries
2. **Triggering condition analysis**: Identify all performance limitations
3. **Extensive scenario database**: Build from real-world data, simulation, and expert knowledge
4. **Validation strategy**: Combine physical testing, simulation, and field monitoring
5. **Continuous improvement**: Update based on field experience

### When to Apply SOTIF

- ✓ Systems with complex AI/ML algorithms
- ✓ Sensor-based perception systems
- ✓ Autonomous driving features (L2-L5)
- ✓ Systems with large scenario variability
- ✓ When ODD cannot cover all possible situations

### Next Steps

- **Notebook 13**: ISO/IEC 8800 - AI-specific safety principles
- **Notebook 14**: ISO/SAE 21434 - Cybersecurity for automotive
- **Exercise 6**: Perform SOTIF analysis for your perception system

---

## References

- ISO/PAS 21448:2019 - Road vehicles — Safety of the intended functionality
- ISO 26262:2018 - Road vehicles — Functional safety
- UL 4600 - Standard for Safety for the Evaluation of Autonomous Products
- "SOTIF: Safety of the Intended Functionality" - SAE International
- "Scenario-Based Testing for Autonomous Vehicles" - Waymo Safety Report

---

**End of Notebook 12**